In [1]:
import os
import json
import re

In [2]:
CLI_OUTPUT_DIR = "/local/juro4948/data/egoexo4d/egoexo" # Replace with the full path to the --output_directory you pass to the cli
ANNOTATIONS_PATH = os.path.join(CLI_OUTPUT_DIR, "annotations")

# See raw annotations in this dictionary
keystep_anns = json.load(open(os.path.join(ANNOTATIONS_PATH, "keystep_train.json")))
keystep_anns_val = json.load(open(os.path.join(ANNOTATIONS_PATH, "keystep_val.json")))

anns = keystep_anns["annotations"]
print(f'Length of training items: {len(anns)}')
anns_test = keystep_anns_val["annotations"]

# Add anns_test to anns dictionary
anns.update(anns_test)
print(f'Length of training + val items: {len(anns)}')

def get_take_id_from_name(take_name):
    for take_id in anns.keys():
        if anns[take_id]['take_name'] == take_name:
            return take_id
    return None

def get_take_name_from_id(take_id):
    if take_id in anns.keys():
        return anns[take_id]['take_name']
    else:
        return None
    

Length of training items: 668
Length of training + val items: 850


In [6]:
len(all_features_fn)

5383

In [7]:
# create symlink to each take 
# assuming that same take's aria and cam are all the in the same split 
import numpy as np
import os
import shutil

### PARAMETERS ###
use_aria = False # True = use aria, False = use all gopro views
annotations_dir_name = 'egoexo-regular' #'egoexo-omnivore-aria-for-mamba' # name of annotations folder in GraVi-T repo so that it can read splits
out_dir_name = 'omnivore-ego-exo_raw' # -exo #'egoexo-omnivore-aria-for-mamba' # name of the output features directory in Gravit/data/features

# path to npy egoexo features
# /home/juro4948/gravit/data/egoexo4d/egoexo4d/preprocessed_old/egoexo4d_omnivore_segmentwise_features_npy
# change this for segmentwise features vs non-segmentwise features
nonsegmentwise = '/local/juro4948/data/egoexo4d/preprocessed_old/egoexo4d_features_npy' # may need to change this
# segmentwise = '/local/juro4948/data/egoexo4d/egoexo4d/preprocessed_old/egoexo4d_omnivore_segmentwise_features_npy'
features_dir = nonsegmentwise
use_brp_features = False



##################

# path to split bundle files. Make sure you have copied the split bundle files produced from generate_splits.py in the BrP repo.
root = f"/home/juro4948/gravit/GraVi-T/data/annotations/{annotations_dir_name}/splits"
all_features_fn = os.listdir(features_dir)
out_dir = '/home/juro4948/gravit/GraVi-T/data/features/' + out_dir_name

n_splits = [1, 2, 3, 4, 5]
aria_ext = ['_aria.._rgb.npy']
# aria_save_ext = '_aria.npy'
gopro_ext = ['_cam.._0.npy', '_gp.._0.npy']

file_list = []
for n_split in n_splits:
    split_dir = os.path.join(out_dir, f'split{n_split}')
    # if dir already exists write over it
    if os.path.exists(split_dir):
        shutil.rmtree(split_dir)
    os.makedirs(split_dir)
    for mode in ['train', 'test']:
        mode_name = mode
        if mode_name == 'test':
            mode_name = 'val'
        os.makedirs(os.path.join(split_dir, mode_name))
        print(f'Split: {n_split} # Mode: {mode}')

        # read every take name from the split file
        txt_path = f'{mode}.split' + str(n_split) + '.bundle'
        with open(os.path.join(root, txt_path), 'r') as f:
            for line in f.readlines():
                line = line.strip('\n')
                line = line.split('.')[0]

                take_name = line 
                take_id = get_take_id_from_name(take_name)
                if take_id is None:
                    print('ID not found. Skipping')
                    quit()


                # symlink the pt file
                if use_aria == True:
                    pattern = re.compile(take_id + aria_ext)
                    pattern1 = None
                else:
                    pattern = re.compile(take_id + gopro_ext[0])
                    pattern1 = re.compile(take_id + gopro_ext[1])

                i = 0
                # find all files with matching pattern (finding all views of the same take that correspond to aria vs gopro), and save each symlink
                for fn in all_features_fn:
                    if use_brp_features:
                        if pattern.match(fn) or (pattern1 is not None and pattern1.match(fn)): 
                            src = os.path.join(features_dir, fn)
                            dst = os.path.join(split_dir, mode_name, take_name + '_' + str(i) + '.npy')
                            i += 1

                            if os.path.exists(dst):
                                continue
                            if not os.path.exists(src):
                                print(f'File not found: {src}')
                                
                            os.symlink(src, dst)
                            print('Creating symlink')
                    else:
                        if pattern.match(fn) or (pattern1 is not None and pattern1.match(fn)): 
                            
                            src = os.path.join(features_dir, fn)
                            dst = os.path.join(split_dir, mode_name, take_name + '_' + str(i) + '.npy')
                            i += 1
                            print(f'Matched: {fn}    |  Dest: {dst}')

                            if os.path.exists(dst):
                                continue
                            if not os.path.exists(src):
                                print(f'File not found: {src}')

                            # copy file instead of symlink
                            # shutil.copy(src, dst)

                            array1 = np.load(src)
                            
                                
                            os.symlink(src, dst)
                            print('Creating symlink')


Split: 1 # Mode: train
Matched: fb626925-969e-48ea-aeda-ef99300fb8f7_gp05_0.npy    |  Dest: /home/juro4948/gravit/GraVi-T/data/features/omnivore-ego-exo_raw/split1/train/upenn_0712_Cooking_5_3_0.npy
Creating symlink
Matched: fb626925-969e-48ea-aeda-ef99300fb8f7_gp06_0.npy    |  Dest: /home/juro4948/gravit/GraVi-T/data/features/omnivore-ego-exo_raw/split1/train/upenn_0712_Cooking_5_3_1.npy
Creating symlink
Matched: fb626925-969e-48ea-aeda-ef99300fb8f7_gp01_0.npy    |  Dest: /home/juro4948/gravit/GraVi-T/data/features/omnivore-ego-exo_raw/split1/train/upenn_0712_Cooking_5_3_2.npy
Creating symlink
Matched: fb626925-969e-48ea-aeda-ef99300fb8f7_gp04_0.npy    |  Dest: /home/juro4948/gravit/GraVi-T/data/features/omnivore-ego-exo_raw/split1/train/upenn_0712_Cooking_5_3_3.npy
Creating symlink
Matched: fb626925-969e-48ea-aeda-ef99300fb8f7_gp02_0.npy    |  Dest: /home/juro4948/gravit/GraVi-T/data/features/omnivore-ego-exo_raw/split1/train/upenn_0712_Cooking_5_3_4.npy
Creating symlink
Matched: fb6

In [4]:
split_dirs = [os.path.join(out_dir, f'split{n_split}') for n_split in n_splits]
file_counts = [len(os.listdir(split_dir)) for split_dir in split_dirs]

file_counts

[2, 2, 2, 2, 2]